In [12]:
import sys
import os
import time
import operator
import pandas as pd

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index  
    
    
start=time.time()

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=4096
Top=[]
#item_count=100000

with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            print('EOF')
            break
        else:
            item=Tail(element,1)
            #item_count-=1
            # print("read {}th element: {}".format(item_count,element))
            if len(Top)==0:
                Top.append(item)
            else:
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                    else:
                        # replace last element with count 
                        Top[-1].ID=item.ID
                        Top[-1].count+=1
                    Top.sort(key=operator.attrgetter('count'),reverse=True)
                else:
                    Top[index].count+=1
                    Top.sort(key=operator.attrgetter('count'),reverse=True)

end=time.time()
print(Top[:20],len(Top))
print("Top-{}".format(size))
print("Total memory {0} bytes=Top-{1} with size {0} bytes.".format(sys.getsizeof(Top),size))
print("Execution time:{:8.3f} seconds.".format(end-start))

#====================result compare=============================
#　conver Top into df    
templi=[[i.ID,i.count] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])

path='..\\result\\kosarak\\'
name="SS_kosarak"+'_'+str(size)
df.to_csv(path+name+".csv",index=False)

#====================result compare=============================
groundtruth='kosarak_ground_truth.csv'
final=name+".csv"
grtruth=pd.read_csv(os.path.join(path,groundtruth))
    # compare with Top-k and groundtruth[k]
My_result=pd.read_csv(os.path.join(path,final))

# precision
gt_set=set(grtruth['Element'][:size])
    # Top-size of ground truth
my_set=set(My_result['ID'])
precision=len(gt_set & my_set)/len(my_set)
    # &: set 交集運算
        
# ARE/AAE
gt_dict=dict(grtruth.values.tolist())
my_dict=dict(My_result.values.tolist())
distinct=len(gt_dict)
are_error=0
aae_error=0
tp=0
fp=0

for item in gt_dict:
    # Overall ARE/AAE
    if item in my_dict:
        # ARE/AAE for Top
        are_error+=abs(my_dict[item]-gt_dict[item])/my_dict[item]
        aae_error+=abs(my_dict[item]-gt_dict[item])
        if my_dict[item]==gt_dict[item]:
            tp+=1
        else:
            fp+=1
            
ARE=are_error/distinct
AAE=aae_error/distinct
print("Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp))
print("Precision: {:8.4f}".format(precision))
print("ARE: {:8.6f}".format(ARE))
print("AAE: {:8.6f}".format(AAE))

EOF
['3', count: 36133, '6', count: 30225, '6 3', count: 21139, '11 6', count: 12769, '1', count: 9766, '11 6 3', count: 9083, '1 3', count: 6002, '11', count: 5374, '1 6', count: 5046, '4', count: 4290, '11 3', count: 3756, '55', count: 3561, '1 6 3', count: 3510, '4 3', count: 3000, '55 3', count: 2377, '11 1 6', count: 2375, '4 6', count: 2340, '4 6 3', count: 1758, '11 1 6 3', count: 1753, '6 55', count: 1749] 4096
Top-4096
Total memory 33928 bytes=Top-4096 with size 33928 bytes.
Execution time:1856.348 seconds.
Find:663, TP:103, FP:3993
Precision:   0.1619
ARE: 0.006221
AAE: 1.202578
